In [3]:
print("hi")

hi


In [ ]:
!pip show llama-index

In [ ]:
!pip install llama-index==0.12.0

In [8]:
import llama_index
help(llama_index)

Help on package llama_index:

NAME
    llama_index

PACKAGE CONTENTS
    _bundle (package)
    cli (package)
    core (package)
    legacy (package)

SUBMODULES
    embeddings
    llms

FILE
    (built-in)




In [7]:
# import os
# from dotenv import load_dotenv
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
# from llama_index.llms.groq import Groq
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [1]:
import requests
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

d:\chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# Set up GROQ API key
GROQ_API_KEY = "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"
GROQ_API_URL = "https://api.groq.io/v1/query"

In [ ]:
# Function to query the GROQ API
def query_groq_api(query):
    headers = {
        "Authorization": GROQ_API_KEY,
        "Content-Type": "application/json"
    }
    payload = {
        "query": query
    }
    response = requests.post(GROQ_API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.text}

# Set up LlamaIndex with new Settings
llm = Groq(model="llama-3.3-70b-versatile")
node_parser = SimpleNodeParser()

# Configure global settings with both LLM and embedding model
Settings.llm = llm
Settings.node_parser = node_parser
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"  # This is a good default model for embeddings
)

# Create index with local config
index = VectorStoreIndex.from_documents(
    documents=[],  # empty list of documents
    llm=llm  # passing llm directly to the index
)

# Define chatbot responses
def chatbot_response(user_input):
    if "fetch data" in user_input.lower():
        # Example GROQ API query
        groq_query = "*[_type == 'dataset']{title, description}"
        api_response = query_groq_api(groq_query)
        
        if "error" in api_response:
            return f"Error fetching data: {api_response['error']}"
        else:
            # Extract and format data from GROQ API response
            data = api_response.get("result", [])
            if not data:
                return "No data found for the query."
            response = "Here are the results:\n"
            for item in data:
                response += f"Title: {item.get('title', 'N/A')}\nDescription: {item.get('description', 'N/A')}\n\n"
            return response
    else:
        return "I can help you fetch data using the GROQ API. Type 'fetch data' to get started!"

# Sample chatbot conversation
def run_chatbot():
    print("Chatbot: Hello! How can I assist you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye!")
            break
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")

# Run chatbot
run_chatbot()